# Как можно оценивать RAG

![lllm_eval.png](../imgs/lllm_eval.png)

## Мотивация
Хочется иметь понимание насколько модификации системы/модели лучше baseline решение.

P.S. Важно определить, что важнее precision(правильно отвечать на вопросы) или recall(отвечать на как можно больше вопросов)?

## Есть несколько этапов, на которых можно оценивать качество RAG
Будем рассмативать этапы с конца
- Генерация
- Ретрив (поиск чанков/документов)

![rag_schema](../imgs/rag_schema.png)

## Метрики генерации
**Можно разбить на 2 группы:**
- Метрики качества итогового ответа (BERTScore и т.д.)
- Метрики качества генерации на основе контекста (Faithfulness, Answer relevancy)

### Качество итогового ответа
Для этих метрик нужны ground truth (эталонные) ответы. Данные метрики показывают насколько получнный ответ схож с ground truth (эталонным) ответом.

**Можно разбить на 3 группы:**
- Non-LLM-Based метрики
    - BertScore
    - Semantic Similarity
    - BLEU (Bilingual Evaluation Understudy)
    - ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
    - METEOR (Metric for Evaluation of Translation with Explicit ORdering)
    - Расстояние Левенштейна
- LLM-Based метрики
    - G-Eval
- Ручная разметка (ассесоры)

#### Non-LLM-Based метрики

##### BertScore
Использует контекстуальные эмбеддинги, учитывает семантическое сходство отличие от BLEU или ROUGE. BERTScore позволяет выявлять эквивалентность смысла даже при различии в словах и формулировках, учитывая синонимы и перефразы. 

Для расчета нужны эталонные ответы и ответы от RAG/LLM/системы, чьи ответы проверяем.

![bertscore](../imgs/bertscore.png)

###### Методика расчета
Метод BERTScore состоит из нескольких этапов:
1. **Получение контекстуальных эмбеддингов:** Оба текста (референсный и сгенерированный) разбиваются на токены и пропускаются через предобученную трансформерную модель (например, BERT или RoBERTa). Для каждого токена извлекается его контекстуальное векторное представление (эмбеддинг).
2. **Вычисление косинусного сходства:** Для всех пар токенов из двух текстов вычисляется косинусное сходство, и формируется матрица сходства токенов.
3. **Расчёт точности, полноты и $F_1$-меры:** На основе матрицы сходства для каждого токена в сгенерированном тексте находится наиболее похожий токен в референсном тексте, что позволяет вычислить точность (precision). Аналогично, для каждого токена референса находится самый близкий токен в сгенерированном тексте, что даёт полноту (recall). Итоговым значением BERTScore является сбалансированная $F_1$-мера, которая комбинирует точность и полноту

> $recall = \frac{1}{|x|} \sum\limits_{x_i \in x} \max\limits_{\hat{x_j} \in \hat{x}} x_i^T \hat{x_j}$

> $precision = \frac{1}{|\hat{x}|} \sum\limits_{\hat{x_j} \in \hat{x}} \max\limits_{x_i \in x} x_i^T \hat{x_j}$

> $BertScore = F_1 = 2 * \frac{precision * recal}{precision + recal}$

4. \* можно взвешивать токены по их важности (с помощью IDF-весов) и преобразовывать оценки от 0 до 1 для лучшей интерпретируемости (параметры в BertScorer)

Допустимые значения без нормировки [-1, 1]. Чем ближе BertScore к 1 тем больше полученный ответ совпадает с эталонным ответом.

###### Плюсы и минусы
**Плюсы:**
- **Учёт семантики:** Сравнивает тексты на уровне смысла, учитывая синонимы и перефразы.
- **Высокая корреляция с человеком:** Оценки BERTScore лучше согласуются с человеческими суждениями о качестве текста, чем традиционные метрики

**Минусы:**
- **Вычислительные затраты:** Расчёт на основе эмбеддингов требует значительно больше ресурсов, чем подсчёт n-грамм, и часто требует использования GPU

##### Semantic Similarity
Semantic Similarity оценивает семантическое сходство между сгенерированным ответом и эталонным ответом. В отличии от **BertScore** сравнивают эмбеддинги не слов/токенов, а ответов/предложений и уже от них берется косинусное расстояние.

Для расчета нужны эталонные ответы и ответы от RAG/LLM/системы, чьи ответы проверяем.

###### Методика расчета
Метод Semantic Similarity состоит из нескольких этапов:
1. **Получение эмбеддингов:** Оба текста (референсный и сгенерированный) пропускаются через предобученную трансформерную модель. Для текстов извлекаются векторные представления (эмбеддинги).
2. **Вычисление косинусного сходства**

Допустимые значения без нормировки [-1, 1]. Чем ближе Semantic Similarity к 1 тем больше полученный ответ совпадает с эталонным ответом.

P.S. В [RAGAS](https://github.com/vibrantlabsai/ragas/blob/main/src/ragas/metrics/collections/_semantic_similarity.py) почему-то написано, что от 0 до 1, хотя считают косинусное расстояние.

###### Плюсы и минусы
**Плюсы:**
- **Учёт семантики:** Сравнивает тексты на уровне смысла, учитывая синонимы и перефразы.
- **Высокая корреляция с человеком** 

**Минусы:**
- **Вычислительные затраты:** Расчёт на основе эмбеддингов требует значительно больше ресурсов, чем подсчёт n-грамм, и часто требует использования GPU


##### BLEU (Bilingual Evaluation Understudy)
BLEU вычисляет точность для каждого совпадающего n-грамма между полученным ответом и эталонным ответом, чтобы найти их геометрическое среднее, и применяет штраф за краткость, если это необходимо. Изначально применялся для оценки машинного перевода.

###### Методика расчета
Метод BLEU состоит из нескольких этапов:
1. **Модифицированная точность n-грамм:** Сначала для n-грамм разной длины (обычно от 1 до 4) рассчитывается их точность ($p_n$) — доля n-грамм из полученного ответа, которые встречаются в эталонном ответе. При этом количество совпадений для каждой n-граммы ограничивается максимальным числом её вхождений в любом из эталонных текстов, чтобы избежать завышения оценки за повторение одного и того же слова.

![bleu_problem](../imgs/bleu_problem.png)

2. **Агрегирование и геометрическое среднее:** Чтобы получить единую оценку, точности для 1-, 2-, 3- и 4-грамм агрегируются с помощью среднего геометрического. Это делается для того, чтобы низкая точность для одного типа n-грамм (например, 4-грамм) сильно влияла на итоговый балл, отражая плохое качество длинных фраз. 
> $Geometric \; average \; precision = \sqrt[4]{p_1 * p_2 * p_3 * p_4} = \prod_{n=1}^{N}p_n^{w_n} = exp(\sum_{n=1}^{N}w_n log(p_n))$

3. **Штраф за краткость (Brevity Penalty):** Чтобы предотвратить получение завышенных оценок за счёт слишком коротких, но точных переводов, BLEU вводит штраф за краткость (Brevity Penalty, BP). Если длина перевода-кандидата (c) существенно меньше длины эталонного перевода (r), итоговый балл BLEU уменьшается. Штраф рассчитывается по формуле:

![brevity_penalty_formula](../imgs/brevity_penalty_formula.png)

4. **Итоговая формула BLEU:** Окончательный BLEU-скор рассчитывается как произведение штрафа за краткость на среднее геометрическое точностей n-грамм:
> $BLEU = BP * Geometric \; average \; precision = BP * exp(\sum_{n=1}^{N}w_n log(p_n))$, где N — максимальная длина n-грамм (обычно 4), а $w_n$ — веса (обычно 1/N).

Допустимые значения [0, 1]. Чем ближе BLEU к 1 тем больше полученный ответ совпадает с эталонным ответом.

###### Плюсы и минусы
**Плюсы:**
- **Скорость расчета и отсутствие потребности в мощностях**

**Минусы:**
- **Отсутствие семантической оценки:** BLEU измеряет лишь поверхностное совпадение слов и не способен оценить, совпадает ли смысл полученного ответа с эталонным.
- **Игнорирование синонимов и перефразов:** Алгоритм наказывает ответы, использующие синонимы или иные формулировки, чем в эталоне, даже если они полностью корректны. Текст может быть точным по смыслу, но написан другими словами — и BLEU его «накажет». 

##### ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
В отличие от метрики BLEU, которая оценивает точность (precision), ROUGE фокусируется на полноте (recall) — она показывает, какая часть значимых фрагментов из эталонного ответа была воспроизведена в сгенерированном тексте. ROUGE изначальна предназначена для оценки суммаризации. Семейство ROUGE включает несколько модификаций: 
- **ROUGE-N** классическая версия, измеряющая совпадения n-грамм (например, ROUGE-1 для отдельных слов, ROUGE-2 для пар слов). 
- **ROUGE-L**  основана на вычислении наибольшей общей подпоследовательности (Longest Common Subsequence), то есть определяет, насколько порядок слов в тексте модели соответствует эталону.
- **ROUGE-S** измеряет совпадения пропущенных биграмм (skip-bigrams), позволяя учитывать слова, стоящие на расстоянии друг от друга. 

Эти модификации возникли потому, что суммаризация — не просто повторение слов, а реконфигурация текста. ROUGE-L и ROUGE-S позволяют фиксировать смысловые совпадения даже при перестановке слов или сокращении предложений.

###### Методика расчета
Считается как и BLEU только вместо деления кол-ва n-грамм на кол-во на общее число n-грамм в машинном переводе (для precision), оно делится на общее число n-грамм в эталонном ответе.

Допустимые значения [0, 1]. Чем ближе ROUGE к 1 тем больше полученный ответ совпадает с эталонным ответом.

###### Плюсы и минусы
**Плюсы:**
- **Скорость расчета и отсутствие потребности в мощностях**

**Минусы:**
- **Отсутствие семантической оценки:** ROUGE измеряет лишь поверхностное совпадение слов и не способен оценить, совпадает ли смысл полученного ответа с эталонным.
- **Игнорирование синонимов и перефразов:** Алгоритм наказывает ответы, использующие синонимы или иные формулировки, чем в эталоне, даже если они полностью корректны. Если BLEU «наказывает» за отсутствие совпадений, ROUGE «поощряет» за охват.


##### METEOR (Metric for Evaluation of Translation with Explicit ORdering)
Метод, который оценивает как точность (совпадения n-граммов), так и полноту (перекрытие n-граммов), с учетом порядка слов в выводах LLM и эталонах. METEOR также использует внешние лингвистические базы данных, такие как WordNet, для учета синонимов. Итоговый балл вычисляется как гармоническое среднее точности и полноты с учетом штрафа за нарушения порядка. Изначальна была предназначена для перевода.

#### LLM-Based метрики

##### G-Eval/LLM-as-a-Judge
G-Eval сначала генерирует последовательность шагов оценки с использованием метода цепочки рассуждений (Chain of Thoughts, CoTs), а затем применяет сгенерированные шаги для определения итоговой оценки через парадигму заполнения формы (проще говоря, G-Eval требует нескольких компонентов информации для выполнения своей работы). Например, для оценки связности вывода LLM с помощью G-Eval необходимо составить запрос, включающий критерии и текст для оценки, чтобы сначала сгенерировать шаги оценки, а затем с помощью LLM вывести итоговый балл по шкале от 1 до 5, опираясь на эти шаги.

https://deepeval.com/guides/guides-answer-correctness-metric

Для расчета нужны эталонные ответы и ответы от RAG/LLM/системы, чьи ответы проверяем.

###### Плюсы и минусы
**Плюсы:**
- **Учёт семантики:** Способен учитывать семантику выводов.
- **Высокая корреляция с человеком:** Оценки G-Eval лучше согласуются с человеческими суждениями о качестве текста, чем традиционные метрики и BertScore

**Минусы:**
- **Вычислительные затраты:** Это все те же запросы в LLM -> времени на получения скоров понадибится немало.
- **Необходимость написание хорошего промпта**

### Качество генерации на основе контекста
Для этих метрик не нужны ground truth (эталонные) ответы. Данные метрики показывают насколько получнный ответ соответствует вопросу/контексту. Решает фактологические проблемы.

**Метрики:**
- Faithfulness (Answer~Context)
- Answer Relevancy (Answer~Query)

#### Faithfulness
(перевод: Достоверность) Метрика показывает согласованность генеративного ответа к найденному контексту (можно считать в **онлайне** и **не нужна** разметка)

##### Методика расчета
Расчет состоит из нескольких этапов:
1. Необходимо сгенерить несколько ответов на вопрос на основе полученного контекста
2. Через LLM as a Judge для каждого утверждения определить можно ли сделать вывод из полученного контекста
3. Посчитать метрику по формуле:

$Faithfulness = \frac{кол-во \; ответов \; согласованных \; с \; контекстом}{кол-во \; сгенерированных \; ответов}$

Принимает значения от [0, 1], чем ближе к 1 тем более достоверный ответ получен.

#### Answer Relevancy
Метрика показывает насколько сгенерированный ответ соответствует вопросу (можно считать в **онлайне** и **не нужна** разметка). Для данной метрики понадобится не только LLM, но и модель эмбеддер.

##### Методика расчета
Расчет состоит из нескольких этапов:
1. Необходимо сгенерить несколько вопросов на основе полученного ответа(на которые мог бы отвечать полученный ответ). Эти вопросы предназначены для отражения содержания ответа.
2. Вычисляем косинусные расстояния между эмбеддингом вопроса пользователя и каждоым эмбеддингом сгенерированного вопроса.
3. Считаем среднее косинусные расстояния

Принимает значения от [0, 1], чем ближе к 1 тем более достоверный ответ получен.

### Вывод
В первом приближении для оценки генерации можно оценить только BertScore/Semantic Similarity, т.к. они дают баланс между трудозатратой вычисления и схожестью с человеческой оценкой.

Если же есть чуть больше времени, то можно посчитать BLEU/ROUGE, но они имеют ряд своих минусов, из-за которых оценка качества ответов может быть не репрезентативной (штрафы за синонимы/перефраз).

Если уже много времени и ресурсов можно воспользоваться G-Eval, Faithfulness и Answer Relevancy, которые дадут более репрезентативную оценку качеству ответов.

## Метрики ретрива
Можно разбить на 2 группы:
- Метрики качества релевантности контекста (Context Precision, Context Recall)
- Метрики оценки качества ретривера (Recall@k, Precision@k, MAP@k и т.д.)

### Качество релевантности контекста
Оценивают, насколько хорошо система находит нужные документы (чанки) в базе знаний. Важно, т.к. плохой поиск гарантирует плохой финальный ответ.

**Метрики:**
- Context Precision
- \* Context Utilization 
- Context Recall

#### Context Precision
Метрика, которая оценивает способность ретривера ранжировать соответствующие фрагменты выше, чем нерелевантные для данного запроса в извлеченном контексте. "Штрафует" за то, что релевантные документы находятся не на первых позициях.

Он рассчитывается на основе Precision@k. Precision@k - это отношение количества релевантных чанков в топ-k к общему кол-ву чанков в топ-k. 

Нужен эталонный ответ.

##### Методика расчета
1. Считаем Precision@k, отношение кол-ва релевантных полученных чанков в топ-k к общему кол-ву полученных чанков в топ-k. 

> $Precision@k = \frac{TP@k}{TP@k + FP@k} = \frac{кол-во \; релевантных \; полученных \; чанков \; в \; топ-k}{кол-во \; полученных \; чанков \; в \; топ-k}$

2. Считаем для каждой k-ой позиции(чанка) в выдаче Precision@k и $v_k$ (релевантность чанка, индикатор $\in$ {0, 1})

> $Context \; Precision@K = \frac{\sum_{k=1}^{K}(Precision@k * v_k)}{кол-во \; релевантных \; документов \; в \; топ-k}$

Релевантность документа определяется при помощи подхода LLM as a Judge, для этого используется входной запрос(вопрос от пользователя), эталонный ответ, чанки для контекста.

Принимает значения [0, 1], чем ближе к 1 тем лучше отранижорваны и релевантны чанки к вопросу.

#### Context Utilization
Единственное отличие от Context Precision в том, что для определения релевантности документа используется не эталонный ответ, а ответ от RAG.

Эталонный ответ не нужен.

##### Методика расчета
Такая же как и у Context Precision

Релевантность документа определяется при помощи подхода LLM as a Judge, для этого используется входной запрос(вопрос от пользователя), ответ от RAG, чанки для контекста.

Принимает значения [0, 1], чем ближе к 1 тем лучше отранижорваны и релевантны чанки к вопросу.

#### Context Recall
Оценивает полноту ретривера: сколько документов/чанков нашла система из всех документов/чанков, которые содержат информацию для ответа на вопрос? Это противоположность метрикам точности — здесь важно ничего не упустить.

Нужен эталонный ответ.

##### Методика расчета
$Context \; Recall = \frac{Кол-во \; релевантных \; документов \; в \; выдаче}{Общее \; кол-во \; релевантных \; документов \; в \; базе \; знаний}$

Релевантность документа определяется при помощи подхода LLM as a Judge, для этого используется входной запрос(вопрос от пользователя), эталонный ответ, чанки для контекста.

Принимает значения [0, 1], чем ближе к 1 тем больше релевантных чанков мы получаем.

## Фреймворки
- [BertScore](https://github.com/Tiiiger/bert_score) - для расчета bertscore
- [RAGAS](https://docs.ragas.io/en/latest/concepts/metrics/) - для метрик: 
    - Semantic Similarity
    - BLEU (Bilingual Evaluation Understudy)
    - ROUGE (Recall-Oriented Understudy for Gisting Evaluation)
    - METEOR (Metric for Evaluation of Translation with Explicit ORdering)
- [deepeval](https://deepeval.com) - G-Eval

## Литература
- [LLM Evaluation Metrics to Explore](https://www.analyticsvidhya.com/blog/2025/03/llm-evaluation-metrics/#h-importance-of-llm-evaluation)
- [[Хабр] Полное руководство по оценке компонентов системы RAG: что необходимо знать](https://habr.com/ru/articles/860390/)
- [[Хабр] Метрики оценки LLM: полное руководство по оценке LLM](https://habr.com/ru/articles/873332/)\
- [[Хабр] Эволюция метрик качества машинного перевода](https://habr.com/ru/articles/745642/)
- [[Хабр] Прокачиваем RAG: тестируем техники и считаем их эффективность. Часть 2](https://habr.com/ru/articles/949124/)
- [BertScore arxiv](https://arxiv.org/pdf/1904.09675)
- [BertScore github](https://github.com/Tiiiger/bert_score)
- [RAGAS](https://docs.ragas.io/en/latest/concepts/metrics/)
- [deepeval](https://deepeval.com/guides/guides-rag-evaluation#evaluating-generation)
- [Как оценивать современные RAG-системы?](https://youtu.be/fE3gDFi7tNQ?si=FbgW8avZEXK15vOI)
- [Полное руководство по оценке компонентов системы RAG: что необходимо знать](https://habr.com/ru/articles/860390/)
- [Руководство для начинающих по оценке конвейеров RAG с использованием RAGAS](https://llmarena.team/blog/rukovodstvo-dlya-nachinayushchih-po-ocenke-konvejerov-rag)
- [Evaluation of Retrieval-Augmented Generation: A Survey](https://arxiv.org/pdf/2405.07437)
- [CURSE OF INSTRUCTIONS: LARGE LANGUAGE MODELS CANNOT FOLLOW MULTIPLE INSTRUCTIONS AT ONCE](https://openreview.net/pdf?id=R6q67CDBCH)